In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
data = pd.read_csv("labeledTrainData.tsv", sep='\t')[:2500]

In [3]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [4]:
tokenize('#100 $11000')

['#', '100', '$', '11000']

In [5]:
def stem(word):
    return stemmer.stem(word.lower())

In [6]:
stem('organic')

'organ'

In [7]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [8]:
bag_of_words(['hi'],['how','hi'])

array([0., 1.])

In [9]:
X = data['review']
y = data['sentiment']

In [10]:
all_words = []
data = []
labels = {}

In [11]:
for X_batch,y_batch in tqdm(zip(X,y)):
    labels[y_batch] = y_batch

2500it [00:00, 1801367.46it/s]


In [12]:
labels

{1: 1, 0: 0}

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    y_batch = y_batch+1
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
#     idx += 1
    all_words.extend(new_X)
    data.append([new_X,np.eye(y_batch,len(labels))[y_batch-1]])

2500it [00:09, 259.43it/s]


In [14]:
data[0][1]

array([0., 1.])

In [15]:
# data = sorted(set(data))
all_words = sorted(set(all_words))

In [16]:
X = []
y = []

In [17]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,all_words))
    y.append(tag)

100%|███████████████████████████████████████| 2500/2500 [02:11<00:00, 19.07it/s]


In [18]:
type(tag)

numpy.ndarray

In [19]:
type(data[0][1])

numpy.ndarray

In [20]:
from sklearn.model_selection import *

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.25)

In [23]:
device = 'cuda'

In [24]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).float()

In [25]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        y_batch = int(torch.argmax(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [26]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [27]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.iters = 10
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),512)
        self.linear2 = Linear(512,512)
        self.output = Linear(512,2)
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.output(preds)
        return preds

In [28]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
batch_size = 32
epochs = 1000

In [30]:
import wandb
PROJECT_NAME = 'Bag-of-Words-Meets-Bags-of-Popcorn'

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for i in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[i:i+batch_size].to(device)
        y_batch = y_train[i:i+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


 58%|███████████████████████                 | 578/1000 [06:04<04:34,  1.53it/s]